In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path

Possible Goals: quarter prediction sales ig, transaction type, upazila analysis, loyalty program, 

In [3]:
customer = pd.read_csv(open(Path('data') / 'customer_dim.csv'))

In [4]:
fact_table = pd.read_csv(open(Path('data') / 'fact_table.csv'))

In [5]:
item_dim = pd.read_csv(open(Path('data') / 'item_dim.csv'))

In [6]:
store_dim = pd.read_csv(open(Path('data') / 'store_dim.csv'))

In [7]:
time_dim = pd.read_csv(open(Path('data') / 'time_dim.csv'))

In [8]:
Trans_dim = pd.read_csv(open(Path('data') / 'Trans_dim.csv'))
Trans_dim

,payment_key,trans_type,bank_name
0,P001,cash,NaN
1,P002,card,AB Bank Limited
2,P003,card,Bangladesh Commerce Bank Limited
3,P004,card,Bank Asia Limited
4,P005,card,BRAC Bank Limited
5,P006,card,Citizens Bank Limited
6,P007,card,City Bank Limited
7,P008,card,Community Bank Bangladesh Limited
8,P009,card,Dhaka Bank Limited
9,P010,card,Dutch-Bangla Bank Limited


In [9]:
combined = fact_table\
    .merge(Trans_dim,how= 'left', on = 'payment_key')\
    .merge(customer, how = 'left', on = 'coustomer_key')\
    .merge(time_dim, how = 'left', on = 'time_key')\
    .merge(item_dim, how = 'left', on = 'item_key')\
    .merge(store_dim, how = 'left', on ='store_key')
combined.head()

,payment_key,coustomer_key,time_key,item_key,store_key,quantity,unit_x,unit_price_x,total_price,trans_type,...,year,item_name,desc,unit_price_y,man_country,supplier,unit_y,division,district,upazila
0,P026,C004510,T049189,I00177,S00307,1,ct,35.0,35.0,card,...,2016,M&M Peanut Candy 1.7 oz,Food - Chocolate,35.0,Netherlands,Bolsius Boxmeer,ct,DHAKA,JAMALPUR,ISLAMPUR
1,P022,C008967,T041209,I00248,S00595,1,rolls,26.0,26.0,card,...,2018,Charmin Ultra Bath Tissue,Kitchen Supplies,26.0,Bangladesh,DENIMACH LTD,rolls,DHAKA,DHAKA,CANTONMENT
2,P030,C007261,T03633,I00195,S00496,8,ct,12.5,100.0,card,...,2016,Dole Fruit in Gel Cups 4.3 oz,Food - Healthy,12.5,poland,CHROMADURLIN S.A.S,ct,CHITTAGONG,NOAKHALI,BEGUMGANJ
3,P032,C007048,T084631,I00131,S0086,8,ct,14.0,112.0,card,...,2020,Paper Bowls 20 oz Ultra Strong,Dishware - Bowls,14.0,Bangladesh,DENIMACH LTD,ct,RANGPUR,RANGPUR,KAUNIA
4,P014,C006430,T071276,I00050,S00488,8,cans,8.0,64.0,card,...,2014,Waterloo Sparkling Watermelon 12oz,a. Beverage Sparkling Water,8.0,poland,CHROMADURLIN S.A.S,cans,CHITTAGONG,KHAGRACHHARI,MATIRANGA


combined.groupby('upazila').agg({'total_price':'sum','quantity':'sum', 'date': lambda x: list(x),'bank_name': lambda x: list(x),'trans_type': lambda x: list(x),'item_combined':lambda x: list(x),'division':'max','district':'max'})

In [10]:
grouped = combined[['coustomer_key','date','total_price','quantity','item_combined','upazila']]
.groupby(['upazila','coustomer_key'])
.agg({'total_price':'sum','quantity':'sum','date': lambda x: list(x),'item_combined':lambda x: list(x)})
grouped[len(grouped['date']) > 1]

SyntaxError: invalid syntax (3530709700.py, line 2)

In [11]:
combined['date'] = pd.to_datetime(combined['date'], dayfirst = True)
[combined.groupby('division').count()['payment_key'].idxmax(),combined.groupby('district').count()['payment_key'].idxmax(),combined.groupby('upazila').count()['payment_key'].idxmax()]


['DHAKA', 'DHAKA', 'KOTWALI']

In [ ]:
pd.date